In [ ]:
interp.repositories() ++= Seq(coursier.maven.MavenRepository("https://jitpack.io"))

In [ ]:
import $ivy.`com.github.propi:rdfrules:1.4.3`
import collection._
import com.github.propi.rdfrules.data._
import org.apache.jena.riot.Lang

import com.github.propi.rdfrules.algorithm.amie._
import com.github.propi.rdfrules.algorithm.dbscan._
import com.github.propi.rdfrules.utils._
import com.github.propi.rdfrules.index._
import com.github.propi.rdfrules.rule._

In [ ]:
val yagoFacts = Graph("data/yagoFacts.tsv")
yagoFacts.cache("yagoFacts.cache")

In [ ]:
val yagoFacts = Graph.fromCache("yagoFacts.cache")

In [ ]:
val dataset = Dataset() + yagoFacts
dataset.cache("dataset.cache")

In [ ]:
val dataset = Dataset.fromCache("dataset.cache")

In [ ]:
val types: Map[TripleItem.Uri, Map[TripleItemType, Int]] = dataset.types()
types.foreach(println)

In [ ]:
val histogram: Map[Histogram.Key, Int] = dataset.histogram(predicate = true, `object` = true)

In [ ]:
val index = dataset.index()
index.cache("index.cache")

In [ ]:
val index = Index.fromCache("index.cache", false)

In [ ]:
Debugger() { implicit debugger =>
    
    val miningTask = Amie()
    index.mine(miningTask)
}

In [ ]:
val onlyObjectConstants = RuleConstraint.ConstantsAtPosition(RuleConstraint.ConstantsAtPosition.ConstantsPosition.Object)
val noConstants = RuleConstraint.ConstantsAtPosition(RuleConstraint.ConstantsAtPosition.ConstantsPosition.Nowhere)


val preparedMiningTask = Amie()
.addThreshold(Threshold.MinHeadCoverage(0.01))
.addThreshold(Threshold.MinSupport(2))
.addConstraint(onlyObjectConstants)
.addPattern(
        AtomPattern(predicate = TripleItem.Uri("isMarriedTo"))  // * => isMarriedTo(Any, Any)
)
.addPattern(
        AtomPattern(subject = 'a', predicate = TripleItem.Uri("isCitizenOf"), `object` = 'b') 
    &:  AtomPattern(subject = 'b', predicate = TripleItem.Uri("participatedIn")) 
    //=>: None
    =>: AtomPattern(subject = 'a')
)

In [ ]:
var ruleset = index.mine(preparedMiningTask)
ruleset.cache("rules.cache")

In [ ]:
var ruleset = Ruleset.fromCache("rules.cache")

In [ ]:
var ruleset = index.mine(preparedMiningTask)
    
ruleset = ruleset
.filter(rule => rule.measures(Measure.HeadCoverage).value > 0.02)
.computeConfidence(0.5)
.computePcaConfidence(0.5)
.computeLift()
.sortBy(Measure.HeadCoverage, Measure.Lift)
.makeClusters(DbScan(minNeighbours = 3, minSimilarity = 0.85))
    
ruleset.export("rules.txt")
println(ruleset.size)
ruleset.foreach(println)